# Optimization Methods

Example implementations of four optimization methods in Python: Gradient Descent, Mini-Batch Gradient Descent, Momentum and Adam.

**Notations**: $\frac{\partial J}{\partial a } = $ `da` for any variable `a`.

## 1 - Install Modules

In [ ]:
import numpy as np
import math

## 2 - Gradient Descent
 
$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$

$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$

$\alpha$ is the learning rate

In [ ]:
def update_parameters_with_gd(parameters, grads, learning_rate):

    L = len(parameters) // 2 # number of layers

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate*grads["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate*grads["db" + str(l+1)])
        
    return parameters
    
    """
    Update parameters using one step of gradient descent
    
    Arguments:
    parameters = python dictionary containing parameters to be updated
    grads = python dictionary containing gradients to update each parameters
    learning_rate = the learning rate
    
    Returns:
    parameters = python dictionary containing updated parameters 
    """

## 3 - Mini-Batch Gradient descent

Let's learn how to build mini-batches from the training set (X, Y).

There are two steps:
- **Shuffle**: Create a shuffled version of the training set (X, Y).


- **Partition**: Partition the shuffled (X, Y) into mini-batches of size `mini_batch_size` (here 64).

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    
    np.random.seed(seed)            
    m = X.shape[1]              
    mini_batches = []
        
    # Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches*mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches*mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches
   
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X = input data; shape = (input size, number of examples)
    Y = labels vector; shape (1, number of examples)
    mini_batch_size = size of the mini-batches
    
    Returns:
    mini_batches = list of synchronous (mini_batch_X, mini_batch_Y)
    """
    

## 4 - Momentum

Using momentum can reduce oscillations of the path taken by mini-batch gradient descent. 

In [ ]:
def initialize_velocity(parameters):
    
    L = len(parameters) // 2 # number of layers
    v = {}
    
    # Initialize velocity
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
        
    return v
    
    """
    Initializes the velocity as a python dictionary
    
    Arguments:
    parameters = python dictionary containing parameters

    Returns:
    v = python dictionary containing the current velocity
    """
    

The momentum update rule is, for $l = 1, ..., L$: 

$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}$

$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}$

where L is the number of layers, $\beta$ is the momentum and $\alpha$ is the learning rate.

In [ ]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):

    L = len(parameters) // 2 # number of layers
    
    # Momentum update for each parameter
    for l in range(L):
        
        # compute velocities
        v["dW" + str(l+1)] = beta * v["dW" + str(l+1)] + (1 - beta) * grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta * v["db" + str(l+1)] + (1 - beta) * grads['db' + str(l+1)]
        # update parameters
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * v["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * v["db" + str(l+1)])
        
    return parameters, v
    
    """
    Updates parameters using Momentum
    
    Arguments:
    parameters = python dictionary containing parameters
    grads = python dictionary containing gradients for each parameter
    v = python dictionary containing the current velocity
    beta = momentum hyperparameter
    learning_rate = learning rate
    
    Returns:
    parameters = python dictionary containing updated parameters 
    v = python dictionary containing updated velocities
    """


- Common values for $\beta$ range from 0.8 to 0.999. If you don't feel inclined to tune this, $\beta = 0.9$ is often a reasonable default. 
- Tuning the optimal $\beta$ for your model might need trying several values to see what works best in term of reducing the value of the cost function $J$. 

## 5 - Adam

- Calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction). 
- Calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction). 
- Updates parameters in a direction based on combining information from "1" and "2".

The update rule is, for $l = 1, ..., L$: 

$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_1)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$
where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages. 
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero


In [ ]:
def initialize_adam(parameters) :
    
    L = len(parameters) // 2 # number of layers
    v = {}
    s = {}
    
    # Initialize v, s
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
    
    return v, s
   
    """
    Initializes v and s as two python dictionaries
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v = python dictionary; contains the exponentially weighted average of the gradient
    s = python dictionary; contains the exponentially weighted average of the squared gradient
    """

In [ ]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    
    L = len(parameters) // 2                 # number of layers
    v_corrected = {}                         # Initializing first moment estimate
    s_corrected = {}                         # Initializing second moment estimate
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of gradients
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]

        # Compute bias-corrected first moment estimate
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - beta1**t)

        # Moving average of squared gradients
        s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) * np.square(grads['dW' + str(l+1)])
        s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) * np.square(grads['db' + str(l+1)])

        # Compute bias-corrected second raw moment estimate
        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - beta2**t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - beta2**t)

        # Update parameters
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * (v_corrected["dW" + str(l+1)] / (np.sqrt(s_corrected["dW" + str(l+1)])+epsilon)))
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * (v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)])+epsilon)))

    return parameters, v, s

    """
    Updates parameters using Adam
    
    Arguments:
    parameters = python dictionary containing parameters
    grads = python dictionary containing gradients for each parameter
    v = python dictionary containing Adam variable - moving average of the first gradient
    s = python dictionary containing Adam variable - moving average of the squared gradient
    learning_rate = learning rate
    beta1 = Exponential decay hyperparameter for the first moment estimates 
    beta2 = Exponential decay hyperparameter for the second moment estimates 
    epsilon = hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters = python dictionary containing updated parameters 
    v = python dictionary containing Adam variable - moving average of the first gradient
    s = python dictionary containing Adam variable - moving average of the squared gradient
    """
    